# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [39]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = 'https://country-leaders.onrender.com'

# assign the /status endpoint to another variable called status_url (1 line)
status_url = '/status'

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{status_url}")

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(f"Request is OK ({req.status_code}): \n{req.text}")
else:
    req.status_code

Request is OK (200): 
"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [40]:
# Set the countries_url variable (1 line)
countries_url = '/countries'

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{countries_url}")

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(f"Request status: {req.status_code}:\n{countries}")

Request status: 403:
{'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [41]:
# Set the cookie_url variable (1 line)
cookie_url = '/cookie'

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(f"{root_url}{cookie_url}")
print(f"Request status: {cookies.status_code}:\n{cookies.json()}")

Request status: 200:
{'message': 'The cookie has been created'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [42]:
import json

# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(f"{root_url}{countries_url}", cookies=cookies.cookies)

# display the countries variable (1 line)
countries = json.loads(countries.text)
print(*countries, sep="\n")

fr
be
ma
us
ru


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [43]:
# Set the leaders_url variable (1 line)
leaders_url = '/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}{leaders_url}", cookies=cookies.cookies)

# display the leaders variable (1 line)
print(f"Request status: {leaders.status_code}:\n{leaders.text}")

Request status: 404:
{"message":"Please specify a country"}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [44]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
import random
country = random.choice(countries)

# assign the output to the leaders variable (1 line)
cookies = requests.get(f"{root_url}{cookie_url}")
leaders = requests.get(f"{root_url}{leaders_url}", cookies=cookies.cookies, params={"country": country})

# display the leaders variable (1 line)
leaders.json()


[{'id': 'Q7747',
  'first_name': 'Vladimir',
  'last_name': 'Putin',
  'birth_date': '1952-10-07',
  'death_date': None,
  'place_of_birth': 'Saint Petersburg',
  'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87',
  'start_mandate': '2000-05-07',
  'end_mandate': '2008-05-07'},
 {'id': 'Q23530',
  'first_name': 'Dmitry',
  'last_name': 'Medvedev',
  'birth_date': '1965-09-14',
  'death_date': None,
  'place_of_birth': 'Saint Petersburg',
  'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87',
  'start_mandate': '2008-05-07',
  'end_mandate': '2012-05-07'},
 {'id': 'Q34453',
  'first_name': 'Boris',
  'last_name': 'Yeltsin',
  'birth_date': '1931-02-01',
  'death_date': '2007-04-23',

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines 
import requests
from bs4 import BeautifulSoup
import json

root_url = 'https://country-leaders.onrender.com'

cookies = requests.get(f"{root_url}/cookie")
countries = requests.get(f"{root_url}/countries", cookies=cookies.cookies).json()

leaders_per_country = {}

for c in countries:
    leaders = requests.get(f"{root_url}/leaders", cookies=cookies.cookies, params={"country": c}).json()

    wikipedia = []
    for l in leaders:
        wikipedia.append(l['wikipedia_url'])
    
    leaders_per_country[c] = wikipedia


print(leaders_per_country)



In [ ]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
# < 15 lines
def get_leaders():

    root_url = 'https://country-leaders.onrender.com'

    cookies = requests.get(f"{root_url}/cookie")
    countries = requests.get(f"{root_url}/countries", cookies=cookies.cookies).json()

    leaders_per_country = {}

    for c in countries:
        leaders = requests.get(f"{root_url}/leaders", cookies=cookies.cookies, params={"country": c}).json()

        wikipedia = []
        for l in leaders:
            wikipedia.append(l['wikipedia_url'])
        
        leaders_per_country[c] = wikipedia

    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
import requests
from bs4 import BeautifulSoup
import json
leaders_per_country = get_leaders()
leaders_per_country

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [34]:
# 3 lines
# Print json data using loop

l = requests.get("https://fr.wikipedia.org/api/rest_v1/page/summary/Charles_de_Gaulle").json()
for k, v in l.items(): 
    print(f"{k}: {v}")


type: standard
title: Charles de Gaulle
displaytitle: <span class="mw-page-title-main">Charles de Gaulle</span>
namespace: {'id': 0, 'text': ''}
wikibase_item: Q2042
titles: {'canonical': 'Charles_de_Gaulle', 'normalized': 'Charles de Gaulle', 'display': '<span class="mw-page-title-main">Charles de Gaulle</span>'}
pageid: 1019357
thumbnail: {'source': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/De_Gaulle-OWI.jpg/330px-De_Gaulle-OWI.jpg', 'width': 320, 'height': 473}
originalimage: {'source': 'https://upload.wikimedia.org/wikipedia/commons/2/27/De_Gaulle-OWI.jpg', 'width': 2480, 'height': 3663}
lang: fr
dir: ltr
revision: 226393752
tid: 7fea5e49-45d2-11f0-9c45-248a1edad818
timestamp: 2025-06-10T08:11:24Z
description: militaire et homme d'État français
description_source: central
content_urls: {'desktop': {'page': 'https://fr.wikipedia.org/wiki/Charles_de_Gaulle', 'revisions': 'https://fr.wikipedia.org/wiki/Charles_de_Gaulle?action=history', 'edit': 'https://fr.wikipedia.o

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [30]:
# 3 lines
from bs4 import BeautifulSoup

r = requests.get('https://fr.wikipedia.org/wiki/Nicolas_Sarkozy')
soup = BeautifulSoup(r.text, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Nicolas Sarkozy — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [31]:
paragraphs = soup.find_all("p")
print(*paragraphs, sep="\n")


<p class="mw-empty-elt">
</p>
<p>« Sarkozy » redirige ici. Pour les autres significations, voir <a class="mw-disambig" href="/wiki/Sarkozy_(homonymie)" title="Sarkozy (homonymie)">Sarkozy (homonymie)</a>, <a class="mw-disambig" href="/wiki/Nagy" title="Nagy">Nagy</a> et <a class="mw-disambig" href="/wiki/Boc%C8%99a" title="Bocșa">Bocșa</a>.
</p>
<p class="mw-empty-elt">
</p>
<p><b>Nicolas Sarközy de Nagy-Bocsa</b><sup class="reference" id="cite_ref-6"><a href="#cite_note-6"><span class="cite-bracket">[</span>d<span class="cite-bracket">]</span></a></sup>, dit <b>Nicolas Sarkozy</b> (<span class="API nowrap" style="font-family:'Segoe UI','DejaVu Sans','Lucida Grande','Lucida Sans Unicode','Arial Unicode MS','Hiragino Kaku Gothic Pro',sans-serif;" title="Alphabet phonétique international">/<a class="mw-redirect" href="/wiki/API_n" title="API n"><span title="[n] en français « n » dans « nous » ; ailleurs avec la langue un peu plus en arrière.">n</span></a><a class="mw-redirect" href="/wik

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [32]:
# <10 lines
for p in paragraphs:
    if p.contents and p.contents[0].name == "b":
        first_paragraph = p
        break

print(first_paragraph.text)

Nicolas Sarközy de Nagy-Bocsa[d], dit Nicolas Sarkozy (/ni.kɔ.la saʁ.kɔ.zi/[e] ; Écouterⓘ), né le 28 janvier 1955 à Paris 17e (Seine), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012. Le 18 décembre 2024, il est le premier président de la Ve République condamné définitivement à de la prison ferme, pour corruption et trafic d'influence dans l'affaire Bismuth, sa condamnation étant confirmée par la cour de cassation.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [80]:
# 10 lines
def get_first_paragraph(wikipedia_url):

    print(wikipedia_url) # keep this for the rest of the notebook

    r = requests.get(wikipedia_url)
    soup = BeautifulSoup(r.text, "html.parser")

    for p in soup.find_all("p"):
        if p.contents and p.contents[0].name == "b":
            first_paragraph = p
            break

    print(first_paragraph.text)
  
    return first_paragraph.text

In [81]:
# Test: 3 lines
first_paragraph = get_first_paragraph('https://fr.wikipedia.org/wiki/Fernand_Hollande')


https://fr.wikipedia.org/wiki/Fernand_Hollande
Fernand Hollande, né le 21 septembre 1876 à Avesnes-sur-Helpe (Nord) et mort le 22 octobre 1938 à Saint-Quentin (Aisne), est un homme politique français.



### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re
pattern = r'\((.*?)ⓘ\)|\[.*?\]'
first_paragraph = re.sub(pattern, '', first_paragraph)
print(first_paragraph)


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [72]:
# 10 lines
def get_first_paragraph(wikipedia_url):

#    print(wikipedia_url) # keep this for the rest of the notebook
    root_url = 'https://country-leaders.onrender.com'

    cookies = requests.get(f"{root_url}/cookie")
    countries = requests.get(f"{root_url}/countries", cookies=cookies.cookies).json()



    r = requests.get(wikipedia_url)
    soup = BeautifulSoup(r.text, "html.parser")

    for p in soup.find_all("p"):
        if p.contents and p.contents[0].name == "b":
            first_paragraph = p
            break

    first_paragraph = first_paragraph.text
    
    pattern = r'\((.*?)ⓘ\)|\[.*?\]'
    first_paragraph = re.sub(pattern, '', first_paragraph)

    pattern = r'(\xa0)'
    first_paragraph = re.sub(pattern, ' ', first_paragraph)

  
    return first_paragraph


In [ ]:
# Test:
first_paragraph = get_first_paragraph('https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Ho...')
first_paragraph

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [85]:
def get_first_paragraph(wikipedia_url):
        
        try:

            r = requests.get(wikipedia_url)
            #response.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")

            for p in soup.find_all("p"):
                if p.contents and p.contents[0].name == "b":
                    first_paragraph = p
                    break

            first_paragraph = first_paragraph.text
            
            pattern = r'\((.*?)ⓘ\)|\[.*?\]'
            first_paragraph = re.sub(pattern, '', first_paragraph)

            pattern = r'(\xa0)'
            first_paragraph = re.sub(pattern, ' ', first_paragraph)

        
            return first_paragraph
        
        except Exception:
            return ""


In [88]:
# < 20 lines
def get_leaders():

    root_url = 'https://country-leaders.onrender.com'

    cookies = requests.get(f"{root_url}/cookie")
    countries = requests.get(f"{root_url}/countries", cookies=cookies.cookies).json()

    data = []

    for c in countries:
        leaders = requests.get(f"{root_url}/leaders", cookies=cookies.cookies, params={"country": c}).json()

        for l in leaders:
            data.append({
                "country": c,
                "last_name": l.get("last_name"),
                "first_name": l.get("first_name"),
                "wikipedia_url": l.get("wikipedia_url")
            })       

    leaders_per_country = pd.DataFrame(data)
    leaders_per_country["first_paragraph"] = leaders_per_country["wikipedia_url"].apply(get_first_paragraph)
    
    

    return leaders_per_country


In [89]:
# Check the output of your function (2 lines)
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
leaders_per_country = get_leaders()

# test = []
# for i in leaders_per_country.loc[:,'wikipedia_url']:
#     try:
#         p = get_first_paragraph(i)
#         test.append(p)

#     except:
#         test.append("Error")

# test

#leaders_per_country['first_paragraph'] = leaders_per_country['wikipedia_url'].apply(get_first_paragraph)

leaders_per_country

,country,last_name,first_name,wikipedia_url,first_paragraph
0,be,Verhofstadt,Guy,https://nl.wikipedia.org/wiki/Guy_Verhofstadt,Guy Maurice Marie-Louise Verhofstadt (Denderm...
1,be,Leterme,Yves,https://nl.wikipedia.org/wiki/Yves_Leterme,"Yves Camille Désiré Leterme (Wervik, 6 oktober..."
2,be,None,Herman,https://nl.wikipedia.org/wiki/Herman_Van_Rompuy,"Herman Achille graaf Van Rompuy (Etterbeek, 31..."
3,be,Delacroix,Léon,https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix,Léon Fréderic Gustave Delacroix (Sint-Joost-te...
4,be,Carton,Henry,https://nl.wikipedia.org/wiki/Henri_Carton_de_...,Henri Victor Marie Ghislain Carton de Wiart (B...
...,...,...,...,...,...
131,fr,Macron,Emmanuel,https://fr.wikipedia.org/wiki/Emmanuel_Macron,"Emmanuel Macron , né le 21 décembre 1977 à Ami..."
132,ma,None,Mohammed,https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%...,مُحمد السادس بن الحسن الثاني العلوي (مواليد 21...
133,ma,None,Hassan,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9...
134,ma,None,Mohammed,https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%...,محمد الخامس بن يوسف بن الحسن بن محمد بن عبد ال...


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [95]:
# Import necessary libraries to run the code
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re


def get_first_paragraph(wikipedia_url):
        
        try:

            r = requests.get(wikipedia_url)
            soup = BeautifulSoup(r.text, "html.parser")

            for p in soup.find_all("p"):
                if p.contents and p.contents[0].name == "b":
                    first_paragraph = p
                    break

            first_paragraph = first_paragraph.text
            
            pattern = r'\((.*?)ⓘ\)|\[.*?\]'
            first_paragraph = re.sub(pattern, '', first_paragraph)

            pattern = r'(\xa0)'
            first_paragraph = re.sub(pattern, ' ', first_paragraph)

        
            return first_paragraph
        
        except Exception:
            return ""


def get_leaders(root_url):

    #root_url = 'https://country-leaders.onrender.com'

    cookies = requests.get(f"{root_url}/cookie")
    countries = requests.get(f"{root_url}/countries", cookies=cookies.cookies).json()

    data = []

    for c in countries:
        leaders = requests.get(f"{root_url}/leaders", cookies=cookies.cookies, params={"country": c}).json()

        for l in leaders:
            data.append({
                "country": c,
                "last_name": l.get("last_name"),
                "first_name": l.get("first_name"),
                "wikipedia_url": l.get("wikipedia_url")
            })       

    leaders_per_country = pd.DataFrame(data)
    leaders_per_country["first_paragraph"] = leaders_per_country["wikipedia_url"].apply(get_first_paragraph)
    
    

    return leaders_per_country


leaders_per_country = get_leaders('https://country-leaders.onrender.com')
leaders_per_country.to_json('file.json', orient='index', compression='infer')
leaders_per_country.to_csv('leaders_data.csv', index = False)







Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!